# AR6-based forcing. Part 2

Based on `12_make-forcing.py` and `13_radiative-forcing-barchart.py`
in https://github.com/ClimateIndicator/forcing-timeseries

In [ ]:
%cd ..

In [2]:
import pathlib
import numpy as np
import pandas as pd
from scipy.stats import linregress
from mce import MCEExecError
from mce.core import ScenarioBase
from mce.core.forcing_ar6 import RfAllAR6

## Input data

In [10]:
outpath = 'datain/ds_historical.h5'
ds = ScenarioBase(outpath=outpath, mode='a')

[2025-04-18 11:09:33 mce.core] INFO:datain/ds_historical.h5 already exists
[2025-04-18 11:09:33 mce.core] INFO:file datain/ds_historical.h5 opened with mode=a


In [11]:
def func(name, obj):
    desc = obj.attrs.get('description')
    if desc is not None:
        print('{}: {}'.format(name, desc))

gin = ds.file['source/forcing-timeseries']
gin.visititems(func)

aci_cal: Forcing uncertainty ensemble related to aerosol-cloud interactions
ari_emitted: Reference radiative forcing by SLCF species
erf_contrails: Effective radiative forcing of contrails and contrail-induced cirrus from 1930 to 2024
erf_irrigation: Land use forcing due to irrigation
erf_solar: Effective radiative forcing of solar irradiance from -6755 to 2299
erf_volcanic: Effective radiative forcing of volcanic activity from -6755 to 2299
gcp_emissions: Emissions of CO2 from 1750 to 2024
ghg_concentrations: Concentrations of GHGs from 1750 to 2024
sarf_landuse: Land use forcing due to albedo change
skeie_ozone_strat: Reference radiative forcing of stratospheric ozone
skeie_ozone_trop: Reference radiative forcing of tropospheric ozone
slcf_emissions: Emissions of SLCF species from 1750 to 2024
temp_obs: Temperature time series used for temperature feedback of ozone forcing from 1850 to 2023
unc/scale: Scale factors by forcing agent
unc/trend_solar: Uncertainty factor of solar trend


### GHG concentrations

In [83]:
df = pd.DataFrame({
    (k, v.attrs['units']):
    v for k, v in gin['ghg_concentrations'].items()
}).set_index(('time', 'yr'))
df

,C2F6,C3F8,C7F16,C8F18,CCl4,CF4,CFC-11,CFC-112,CFC-112a,CFC-113,...,Halon-2402,N2O,NF3,SF6,SO2F2,c-C4F8,i-C6F14,n-C4F10,n-C5F12,n-C6F14
,ppt,ppt,ppt,ppt,ppt,ppt,ppt,ppt,ppt,ppt,...,ppt,ppb,ppt,ppt,ppt,ppt,ppt,ppt,ppt,ppt
"(time, yr)",,,,,,,,,,,,,,,,,,,,,
1750,0.000001,0.00,0.00000,0.000,0.025,34.05,0.0,0.0,0.0000,0.00,...,0.0,270.100000,0.0,0.000,0.000021,0.0000,0.0000,0.0000,0.0000,0.0000
1850,0.000001,0.00,0.00000,0.000,0.025,34.05,0.0,0.0,0.0000,0.00,...,0.0,272.100000,0.0,0.000,0.000021,0.0000,0.0000,0.0000,0.0000,0.0000
1851,0.000010,0.00,0.00000,0.000,0.025,34.05,0.0,0.0,0.0000,0.00,...,0.0,272.181132,0.0,0.000,0.000021,0.0000,0.0000,0.0000,0.0000,0.0000
1852,0.000021,0.00,0.00000,0.000,0.025,34.05,0.0,0.0,0.0000,0.00,...,0.0,272.262214,0.0,0.000,0.000021,0.0000,0.0000,0.0000,0.0000,0.0000
1853,0.000031,0.00,0.00000,0.000,0.025,34.05,0.0,0.0,0.0000,0.00,...,0.0,272.365269,0.0,0.000,0.000022,0.0000,0.0000,0.0000,0.0000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020,4.940000,0.70,0.11984,0.097,77.000,86.40,223.9,0.4,0.0704,69.20,...,0.4,333.300000,2.3,10.270,2.600000,1.8114,0.0690,0.2073,0.1528,0.2312
2021,5.030000,0.72,0.12132,0.098,76.000,87.40,221.6,0.4,0.0712,68.60,...,0.4,334.600000,2.5,10.650,2.700000,1.8742,0.0704,0.2114,0.1564,0.2328


In [85]:
# interpolation between 1750 and 1850
years = np.arange(df.index[0], df.index[-1]+1)
df = df.reindex(years).interpolate()

In [37]:
g = ds.file.create_group('historical/input/conc')

for k, v in df.reset_index().items():
    d = g.create_dataset(k[0], data=v.values)
    d.attrs['units'] = k[1]

### CO2 emissions

In [86]:
if not np.array_equal(years, gin['gcp_emissions']['time']):
    raise MCEExecError('inconsistent year')

In [ ]:
g = ds.file.create_group('historical/input/emis_co2')

for k, v in gin['gcp_emissions'].items():
    d = g.create_dataset(k, data=v[:])
    d.attrs['units'] = v.attrs['units']

### SLCF emissions

In [89]:
if not np.array_equal(years, gin['slcf_emissions']['time']):
    raise MCEExecError('inconsistent year')

In [ ]:
g = ds.file.create_group('historical/input/emis_slcf')

for k, v in gin['slcf_emissions'].items():
    d = g.create_dataset(k, data=v[:])
    d.attrs['units'] = v.attrs['units']

### ERF of land_use, contrails, solar, and volcanics

In [92]:
d1 = pd.DataFrame({
    (k, v.attrs['units']): v
    for k, v in gin['sarf_landuse'].items() if k in ['time', 'LUH2-GCB2024']
}).set_index(('time', 'yr')).squeeze()

d1 = d1.loc[years[0]:]
d1 *= -0.15 / d1.loc[2004]
d1.loc[2024] = d1.loc[2023]

if not np.array_equal(years, d1.index):
    raise MCEExecError('inconsistent year')

data = [d1.rename(('land_use', d1.name[1]))]

In [93]:
d1 = pd.DataFrame({
    (k, v.attrs['units']): v for k, v in gin['erf_irrigation'].items()
}).set_index(('time', 'yr')).squeeze()
d1

(time, yr)
1750    0.000000
1751    0.000056
1752    0.000109
1753    0.000159
1754    0.000207
          ...   
2018   -0.049217
2019   -0.050000
2020   -0.050549
2021   -0.050624
2022   -0.050749
Name: (value, W m-2), Length: 273, dtype: float64

In [94]:
# Extrapolation
lr = linregress(d1.loc[2013:].index, d1.loc[2013:])
for y1 in [2023, 2024]:
    d1.loc[y1] = lr.slope * y1 + lr.intercept

In [95]:
if not np.array_equal(years, d1.index):
    raise MCEExecError('inconsistent year')

In [96]:
data.append(d1.rename(('irrigation', d1.name[1])))

In [102]:
for cat in ['contrails', 'solar', 'volcanic']:
    d1 = pd.DataFrame({
        (k, v.attrs['units']): v for k, v in gin[f'erf_{cat}'].items()
    }).set_index(('time', 'yr')).squeeze()

    if cat == 'contrails':
        d1 = d1.reindex(years, fill_value=0.)
    else:
        d1 = d1.loc[years[0]:years[-1]]

    if not np.array_equal(years, d1.index):
        raise MCEExecError('inconsistent year')

    data.append(d1.rename((cat, d1.name[1])))

In [105]:
g = ds.file.create_group('historical/input/erf_other')

for k, v in pd.concat(data, axis=1).reset_index().items():
    d = g.create_dataset(k[0], data=v)
    d.attrs['units'] = k[1]

### Categorized input for MCE

In [107]:
dsin = ds.get_scenario('historical')
list(dsin)

['conc', 'emis_co2', 'emis_slcf', 'erf_other']

## Forcing calculation

### Reference data for comparison

In [7]:
path = pathlib.Path('../ClimateIndicator/forcing-timeseries/output/ERF_best_1750-2024.csv')
dfref = {
    'erf_best': pd.read_csv(path, index_col=0),
    'erf_best_agg': pd.read_csv(path.with_stem('ERF_best_aggregates_1750-2024'), index_col=0).rename(int),
    'erf_p05_agg': pd.read_csv(path.with_stem('ERF_p05_aggregates_1750-2024'), index_col=0).rename(int),
    'erf_p95_agg': pd.read_csv(path.with_stem('ERF_p95_aggregates_1750-2024'), index_col=0).rename(int),
}

### Create AR6-based forcing instance

In [108]:
d = dsin['emis_slcf']
df_emis_slcf = pd.DataFrame(d['data'], index=d['time'], columns=d['variables'])

d = dsin['conc']
df_conc = pd.DataFrame(d['data'], index=d['time'], columns=d['variables'])

In [112]:
df_emis_slcf = df_emis_slcf.drop(['CH4', 'N2O'], axis=1)

In [113]:
forcing = RfAllAR6(df_emis_slcf, df_conc)

In [122]:
g = ds.file['source/forcing-timeseries/ari_emitted']
df = pd.DataFrame({k: v for k, v in g.items()}, index=g.attrs['species'])
forcing.init__ari(df['mean'], df['sd'])

In [124]:
g = ds.file['source/forcing-timeseries/aci_cal']
df = pd.DataFrame({k: v for k, v in g.items()}, index=g.attrs['models'])
forcing.init__aci(df)

In [128]:
g = ds.file['source/forcing-timeseries/temp_obs']
args = [pd.Series(g['value'], index=g['time'])]

In [132]:
for k in ['trop', 'strat']:
    g = ds.file[f'source/forcing-timeseries/skeie_ozone_{k}']
    args.append(
        pd.DataFrame(
            g['value'],
            index=g.attrs['index_model'],
            columns=g.attrs['columns_year'],
        )
    )

In [133]:
forcing.init__o3(*args)

In [138]:
df_scale = pd.DataFrame({
    k: v for k, v in ds.file['source/forcing-timeseries/unc/scale'].items()
})

In [140]:
trend_solar = pd.Series(ds.file['source/forcing-timeseries/unc/trend_solar'])

### Best estimate

In [141]:
erf_best = pd.concat([
    forcing.erf__ghg_major(df_conc),
    forcing.erf__ghg_minor(df_conc),
], axis=1)

In [145]:
halogens = [x for x in erf_best.columns if x not in ['CO2', 'CH4', 'N2O']]
len(halogens)

49

In [146]:
cat = 'aerosol-radiation_interactions'
erf_best[cat] = forcing.erf__ari(df_emis_slcf, df_conc)

In [147]:
cat = 'aerosol-cloud_interactions'
erf_best[cat] = forcing.erf__aci(df_emis_slcf)

In [148]:
cat = 'O3'
erf_best[cat] = forcing.erf__o3(df_emis_slcf, df_conc)

In [149]:
cat = 'BC_on_snow'
d1 = df_emis_slcf['BC']
erf_best[cat] = forcing.bc_on_snow__factor * (d1 - d1.loc[1750])

In [150]:
cat = 'H2O_stratospheric'
d1 = df_conc['CH4']
erf_best[cat] = forcing.h2o_strat__factor * (d1 - d1.loc[1750])

In [151]:
d = dsin['erf_other']
df = pd.DataFrame(d['data'], index=d['time'], columns=d['variables'])

cats = ['irrigation', 'land_use']
erf_best = pd.concat([
    erf_best,
    df.drop(cats, axis=1),
    df[cats].sum(axis=1).to_frame('land_use'),
], axis=1)

df__erf_other = df

In [152]:
erf_best.shape, dfref['erf_best'].shape

((275, 61), (275, 61))

In [153]:
np.allclose(
    erf_best,
    dfref['erf_best'][erf_best.columns],
)

True

In [156]:
erf_best_agg = erf_best.drop(halogens, axis=1)

In [157]:
cats = ['aerosol-radiation_interactions', 'aerosol-cloud_interactions']
erf_best_agg['aerosol'] = erf_best[cats].sum(axis=1)

erf_best_agg['halogen'] = erf_best[halogens].sum(axis=1)
erf_best_agg['nonco2wmghg'] = erf_best[['CH4', 'N2O'] + halogens].sum(axis=1)

cats = ['H2O_stratospheric', 'contrails', 'BC_on_snow', 'land_use']
erf_best_agg['minor'] = erf_best[cats].sum(axis=1)

cats = ['solar', 'volcanic']
erf_best_agg['anthro'] = erf_best.drop(cats, axis=1).sum(axis=1)

erf_best_agg['total'] = erf_best.sum(axis=1)

In [158]:
erf_best_agg.shape, dfref['erf_best_agg'].shape

((275, 18), (275, 18))

In [159]:
np.allclose(
    erf_best_agg,
    dfref['erf_best_agg'][erf_best_agg.columns],
)

True

### 90% uncertainty range

In [160]:
dfref_unc = pd.concat({
    'p05': dfref['erf_p05_agg'],
    'p95': dfref['erf_p95_agg'],
}, axis=1).reorder_levels([1, 0], axis=1).sort_index(axis=1)

In [161]:
cats_agg = list(erf_best_agg)

In [162]:
[k for k in cats_agg if k not in erf_best]

['aerosol', 'halogen', 'nonco2wmghg', 'minor', 'anthro', 'total']

In [163]:
ens_agg = {
    k: 0. for k in [
        'aerosol',
        'nonco2wmghg',
        'minor',
        'anthro',
        'total',
    ]
}
ens_agg_count = {
    k: [] for k in [
        'aerosol',
        'nonco2wmghg',
        'minor',
        'anthro',
        'total',
    ]
}
ens_pct = {}

In [164]:
cat = 'aerosol-radiation_interactions'
dfin = forcing.save__ari_dfin
radeff_ens = forcing.ari__radeff_ens[dfin.keys()]
ens = (
    dfin.values[:, None, :] * radeff_ens.values[None, :, :]
).sum(axis=2)
ens_pct[cat] = np.percentile(ens, [5, 95], axis=1).T

for k in ['aerosol', 'anthro', 'total']:
    ens_agg[k] += ens
    ens_agg_count[k].append(cat)

In [165]:
cat = 'aerosol-cloud_interactions'
ens = forcing.save__aci_ens.values
ens_pct[cat] = np.percentile(ens, [5, 95], axis=1).T

for k in ['aerosol', 'anthro', 'total']:
    ens_agg[k] += ens
    ens_agg_count[k].append(cat)

In [166]:
cat = 'land_use'
ens = (
    df__erf_other['irrigation'].values[:, None] * df_scale['irrigation'].values
    + df__erf_other['land_use'].values[:, None] * df_scale['land_use'].values
)
ens_pct[cat] = np.percentile(ens, [5, 95], axis=1).T

for k in ['minor', 'anthro', 'total']:
    ens_agg[k] += ens
    ens_agg_count[k].append(cat)

In [167]:
cat = 'solar'
d1 = pd.Series(np.nan, index=erf_best.index)
d1.loc[1750] = 0.
d1.loc[2019:] = 1.
d1 = d1.interpolate()
ens = (
    erf_best[cat].values[:, None] * df_scale[cat].values
    + d1.values[:, None] * trend_solar.values[None, :]
)
ens_pct[cat] = np.percentile(ens, [5, 95], axis=1).T

ens_agg['total'] += ens
ens_agg_count['total'].append(cat)

In [168]:
cat = 'halogen'
ens = 0.
for k in halogens:
    ens += erf_best[k].values[:, None] * df_scale[k].values

ens_pct[cat] = np.percentile(ens, [5, 95], axis=1).T

for k in [
    'nonco2wmghg',
    'anthro',
    'total',
]:
    ens_agg[k] += ens
    ens_agg_count[k].append(cat)

In [169]:
for cat in cats_agg:
    if cat in ens_pct or cat in ens_agg:
        continue

    ens = erf_best[cat].values[:, None] * df_scale[cat].values
    ens_pct[cat] = np.percentile(ens, [5, 95], axis=1).T

    ens_agg['total'] += ens
    ens_agg_count['total'].append(cat)
    if cat not in ['solar', 'volcanic']:
        ens_agg['anthro'] += ens
        ens_agg_count['anthro'].append(cat)
    if cat in ['CH4', 'N2O']:
        ens_agg['nonco2wmghg'] += ens
        ens_agg_count['nonco2wmghg'].append(cat)
        # 'halogen' counted already
    if cat in ['H2O_stratospheric', 'contrails', 'BC_on_snow']:
        # 'land_use' counted already
        ens_agg['minor'] += ens
        ens_agg_count['minor'].append(cat)

In [170]:
for cat, ens in ens_agg.items():
    ens_pct[cat] = np.percentile(ens, [5, 95], axis=1).T

In [171]:
for i, (cat, pct) in enumerate(ens_pct.items()):
    print(i, np.allclose(pct, dfref_unc[cat]), cat)

0 True aerosol-radiation_interactions
1 True aerosol-cloud_interactions
2 True land_use
3 True solar
4 True halogen
5 True CO2
6 True CH4
7 True N2O
8 True O3
9 True BC_on_snow
10 True H2O_stratospheric
11 True contrails
12 True volcanic
13 True aerosol
14 True nonco2wmghg
15 True minor
16 True anthro
17 True total


In [172]:
ens_agg_count

{'aerosol': ['aerosol-radiation_interactions', 'aerosol-cloud_interactions'],
 'nonco2wmghg': ['halogen', 'CH4', 'N2O'],
 'minor': ['land_use', 'BC_on_snow', 'H2O_stratospheric', 'contrails'],
 'anthro': ['aerosol-radiation_interactions',
  'aerosol-cloud_interactions',
  'land_use',
  'halogen',
  'CO2',
  'CH4',
  'N2O',
  'O3',
  'BC_on_snow',
  'H2O_stratospheric',
  'contrails'],
 'total': ['aerosol-radiation_interactions',
  'aerosol-cloud_interactions',
  'land_use',
  'solar',
  'halogen',
  'CO2',
  'CH4',
  'N2O',
  'O3',
  'BC_on_snow',
  'H2O_stratospheric',
  'contrails',
  'volcanic']}

In [173]:
ds.close()

[2025-04-18 14:45:57 mce.core] INFO:file datain/ds_historical.h5 closed
